# <font color='green'>1. Import needed packages</font>

Additional needed packages:
- imblearn
- tpot

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns
from sklearn import metrics, preprocessing, tree
from sklearn.metrics import make_scorer, SCORERS
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# <font color='green'>2. Importing and understanding the data</font>

In [ ]:
df_labels= pd.read_csv('all-ClinicalLabels-OLD_case_sensitive.csv', low_memory=False)
df_inputs= pd.read_csv('dataset_Elderberry_plus_120nm_64_onh_vessel_removed.csv', low_memory=False)

#display(df_labels) #FULL_ID
#display(df_inputs) #Observations

df_inputs = df_inputs.rename(columns={'Observations': 'FULL_ID'})
merged_df = pd.merge(df_labels, df_inputs, on='FULL_ID', how='inner')

#print(df_labels['FULL_ID'].nunique())
#print(df_inputs['FULL_ID'].nunique())
print(merged_df['PATIENTID'].nunique())

merged_df

In [ ]:
grouped_df = merged_df.groupby(['PATIENTID']).mean()
grouped_df

In [ ]:
counts=grouped_df['PET_RESULT'].value_counts().plot(kind='barh',color='green');
grouped_df['PET_RESULT'].value_counts().plot(kind='barh',color='green')
plt.savefig('plot0.png', dpi=300, bbox_inches='tight',transparent =True)
print(counts)

In [5]:
X = grouped_df.drop(['AGE','HIS_SCORE','PET_RESULT'], axis=1)
y = grouped_df['PET_RESULT']

# ros = RandomUnderSampler(random_state=42)
ros = SMOTE(random_state=42)
X, y = ros.fit_resample(X, y)

In [ ]:
counts2=y.value_counts().plot(kind='barh',color='purple')
y.value_counts().plot(kind='barh',color='purple')
plt.savefig('plot1.png', dpi=300, bbox_inches='tight',transparent =True)
print(counts2)

In [7]:
# grouped_df.value_counts(sort=True, ascending=True).plot(kind='barh',color='green');
# plt.savefig('plot0.png', dpi=300, bbox_inches='tight',transparent =True)
# display(grouped_df['PET_RESULT'].value_counts(sort=True, ascending=True))
# print('Number of different patients: '+str(grouped_df['PATIENTID'].nunique()))

In [8]:
# y.value_counts(sort=True, ascending=True).plot(kind='barh',color='purple');
# plt.savefig('plot1.png', dpi=300, bbox_inches='tight',transparent =True)
# display(y.value_counts(sort=True, ascending=True))
# print('Number of different patients: '+str(X['PATIENTID'].nunique()))

In [9]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# pca = PCA(n_components=18, svd_solver='arpack')
# X=pca.fit_transform(X)
# display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# <font color='green'>5. Automated Classifier and Hyperparameter tunning with TPOT-AutoML</font>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from tpot import TPOTClassifier

# Define TPOT classifier
tpot_model = TPOTClassifier(generations=50, population_size=10, 
                            periodic_checkpoint_folder="tpot_mnst1.txt", verbosity=2, 
                            random_state=42, cv = kf, scoring = 'roc_auc', n_jobs=-1, 
                            config_dict=None) 

# Fit/start training
tpot_model.fit(X_train, y_train)
print('Done training/fitting TPOT session.')

# Get TPOT's score on test set (default metric is 'accuracy'; define something else in TPOT classifier if needed)
print('TPOTs score on test set is...')
print(tpot_model.score(X_test, y_test))

# Export the best pipeline
tpot_model.export('tpot_best_pipeline.py')

# Create sorted by CV (highest to lowest) dataframe 
my_dict = list(tpot_model.evaluated_individuals_.items())
# Create an empty dataframe to append the model strings, model info strings and CV score strings to
model_scores = pd.DataFrame()
for model in my_dict:
    model_name = model[0]
    model_info = model[1] # You could take this out if the values of the pipeline aren't important to you
    cv_score = model[1].get('internal_cv_score')  # Pull out cv_score as a column (i.e., sortable)
    model_scores = model_scores.append({'model': model_name,
                                        'cv_score': cv_score, # You could take this out if the values of the pipeline aren't important to you
                                        'model_info': model_info,},
                                       ignore_index=True)
    
# Sort by best CV score to worst (top to bottom)
model_scores = model_scores.sort_values('cv_score', ascending=False)
print('Model Scores dataframe is...')
print(model_scores)

# Remove duplicate CV score rows and keep top X pipelines (to get best, 'unique' pipelines)
model_scores = model_scores.drop_duplicates(subset ="cv_score", keep = False)

Optimization Progress:   0%|          | 0/510 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8124195849575543

Generation 2 - Current best internal CV score: 0.8124195849575543

Generation 3 - Current best internal CV score: 0.8124195849575543

Generation 4 - Current best internal CV score: 0.8124195849575543

Generation 5 - Current best internal CV score: 0.8124195849575543

Generation 6 - Current best internal CV score: 0.8553148410954792

Generation 7 - Current best internal CV score: 0.8553148410954792

Generation 8 - Current best internal CV score: 0.8553148410954792
